In [161]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import geopandas as gpd


In [2]:
df = pd.read_csv('도서관정보나루_참여도서관목.csv')

In [5]:
lib_df = df.drop(columns = '휴관일')

In [8]:
lib_df.to_csv('data/seoul_library.csv',index = False)

In [11]:
lib_df['지역구'] = lib_df['주소'].apply(lambda x : x.split(' ')[1])
lib_df.groupby('지역구')['주소'].count().reset_index(name = 'lib_cnt')

In [16]:
lib_df.to_csv('data/seoul_library.csv',index = False,encoding = 'cp949')

In [18]:
with pd.ExcelWriter('data/seoul_library.xlsx') as writer:  
    lib_df.to_excel(writer, sheet_name='Sheet_name_1')

# 스마트 도서관

In [39]:
lib_df2 = lib_df[['도서관명','경도','위도','지역구']]
lib_df2['whether'] = 'general'

lib_df2.rename(columns = {'도서관명' : 'location'
                         ,'경도' : 'x'
                         ,'위도' : 'y'
                         ,'지역구' : 'gu'}, inplace = True)

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
smart = pd.read_csv('스마트,U도서관데이터.csv')

In [23]:
smart2 = smart[['location','x','y','gu']]

In [35]:
smart_err = smart2[smart2['x'] == 'error']
smart_non_err = smart2[smart2['x'] != 'error']

In [32]:
x = ['126.8473855','126.8867646','126.9110933','127.026333','127.0349742','126.9624925']
y = ['37.5509103','37.466351','37.487519','37.5114869','37.5172292','37.5545635']

In [33]:
smart_err2 = smart_err[smart_err['location']!= 'error'] 
smart_err2['x'] = x
smart_err2['y'] = y

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
smart_real = pd.concat([smart_non_err,smart_err2],axis = 0)

In [41]:
smart_real['whether'] = 'smart'

In [47]:
smart_real.shape

(52, 5)

In [43]:
lib_final = pd.concat([lib_df2, smart_real],axis = 0)

In [44]:
with pd.ExcelWriter('data/library_final.xlsx') as writer:  
    lib_final.to_excel(writer, sheet_name='Sheet_name_1')

In [106]:
lib_final.rename(columns = {'x' : 'x_coord'
                 ,'y' : 'y_coord'},inplace = True)

In [149]:
temp = lib_final[lib_final['location'] != '서울대입구역']
temp = temp[temp['location'] != '신대방역']
temp = temp[temp['location'] != '봉천역']
temp = temp[temp['location'] != '낙성대역']
temp = temp[temp['location'] != '관악중앙도서관']
temp = temp[temp['location'] != '목동역']
temp = temp[temp['location'] != '쌍문역']

In [ ]:
#temp.groupby(['gu','whether'])['location'].count().reset_index()

In [154]:
temp['whether2'] = temp['whether'].map({'general' : ' ',
                    'smart' : 'smart'})

In [157]:
with pd.ExcelWriter('data/library_final.xlsx') as writer:  
    temp.to_excel(writer, sheet_name='Sheet_name_1')

In [129]:
geo_test = pd.read_csv('data/geo_test.csv')

In [172]:
geo_test

,gu,w_dong,w_lat,w_lng,origin_dong,origin_lat,origin_lng
0,송파구,송파1동,37.506090,127.109571,송파2동,37.502323,127.116741
1,구로구,개봉1동,37.495650,126.887771,개봉1동,37.495650,126.887771
2,금천구,가산동,37.476681,126.883767,독산1동,37.456979,126.895656
3,강서구,가양1동,37.550968,126.849614,가양1동,37.550968,126.849614
4,종로구,종로1.2.3.4가동,37.570962,126.977881,숭인1동,37.573172,126.979220
5,은평구,갈현1동,37.602766,126.928839,갈현1동,37.602766,126.928839
6,강동구,고덕1동,37.530120,127.123858,고덕1동,37.530120,127.123858
7,강남구,역삼2동,37.496040,127.046790,대치4동,37.499750,127.057747
8,마포구,서교동,37.553838,126.918765,서강동,37.547780,126.932218
9,노원구,중계2.3동,37.642040,127.068711,상계6.7동,37.654919,127.066919


In [140]:
geo_test2 = geo_test[['w_dong','w_lng','w_lat','gu']].rename(columns = {'w_dong' : 'location'
                                                                       ,'w_lng' : 'x_coord'
                                                                       ,'w_lat' : 'y_coord'
                                                                       })
geo_test3 = geo_test[['origin_dong','origin_lng','origin_lat','gu']].rename(columns = {'origin_dong' : 'location'
                                                                       ,'origin_lng' : 'x_coord'
                                                                       ,'origin_lat' : 'y_coord'
                                                                       })

In [141]:
geo_test2['whether'] = 'convert'
geo_test3['whether'] = 'middle'

In [176]:
lib_df_fin = pd.concat([lib_final,geo_test2,geo_test3], axis = 0)

In [177]:
lib_df_fin

,location,x_coord,y_coord,gu,whether
0,가락몰도서관,127.112,37.493,송파구,general
1,가온도서관,127.011,37.5564,중구,general
2,갈현1동문화의집,126.917,37.624,은평구,general
3,강남구립논현도서관,127.037,37.5173,강남구,general
4,강남구립대치도서관,127.064,37.4954,강남구,general
...,...,...,...,...,...
20,당산1동,126.896,37.5264,영등포구,middle
21,도봉1동,127.047,37.6686,도봉구,middle
22,망우3동,127.093,37.6067,중랑구,middle
23,신정4동,126.859,37.5234,양천구,middle


In [145]:
with pd.ExcelWriter('data/lib_optimize_ver0.xlsx') as write:
    lib_df_fin.to_excel(write, sheet_name='Sheet_name_1')

# coord to addr

In [203]:
y = lib_df_fin['y_coord'].tolist()
x = lib_df_fin['x_coord'].tolist()
whether = lib_df_fin['whether'].tolist()
idx_ls = list(range(len(x)))

In [204]:
lib_df_fin2 = pd.DataFrame()
for lon, lat, idx in zip(x,y,idx_ls):
    
    if idx % 100 == 0:
        print('처리 : ', idx)
    try:
        
        base_url = f'http://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point={lon},{lat}&format=xml&type=PARCEL&zipcode=false&simple=false&key=66D37555-E9A7-3294-9C22-28B005924F38'

        req = requests.get(base_url)
        soup = BeautifulSoup(req.content,'html.parser')
        body = soup.select('response > result > item')[0]

        total = body.select('text')[0].text

        deep = body.select('structure')[0]

        dong0 = deep.select('level2')[0].text
        dong1 = deep.select('level4l')[0].text
        dong2 = deep.select('level4lc')[0].text
        dong3 = deep.select('level4a')[0].text
        dong4 = deep.select('level4ac')[0].text

        dong_df0 = pd.DataFrame({'x_coord':[lon]
                                ,'y_coord':[lat]
                                ,'total_addr':[total]
                                ,'city':[dong0]
                                ,'dong1':[dong1]
                                ,'dong1_code':[dong2]
                                ,'dong2':[dong3]
                                ,'dong2_code':[dong4]
                                ,'whether' : [whether[idx]]})

        lib_df_fin2 = lib_df_fin2.append(dong_df0)
    except:
        dong_df0 = pd.DataFrame({'x_coord':[lon]
                                ,'y_coord':[lat]
                                ,'total_addr':['err']
                                ,'city':['err']
                                ,'dong1':['err']
                                ,'dong1_code':['err']
                                ,'dong2':['err']
                                ,'dong2_code':['err']
                                ,'whether' : [whether[idx]]})
        lib_df_fin2 = lib_df_fin2.append(dong_df0)

처리 :  0
처리 :  100
처리 :  200
처리 :  300


In [61]:
lib_df

,x_coord,y_coord,total_addr,city,dong1,dong1_code,dong2,dong2_code
0,127.112,37.493,서울특별시 송파구 가락동 600,송파구,가락동,1171010700,가락1동,1171063100
0,127.011,37.5564,서울특별시 중구 신당동 425-11,중구,신당동,1114016200,다산동,1114062500
0,126.917,37.624,서울특별시 은평구 갈현동 436-51,은평구,갈현동,1138010400,갈현제1동,1138055100
0,127.037,37.5173,서울특별시 강남구 논현동 114-24,강남구,논현동,1168010800,논현2동,1168053100
0,127.064,37.4954,서울특별시 강남구 대치동 316,강남구,대치동,1168010600,대치2동,1168061000
...,...,...,...,...,...,...,...,...
0,126.8867646,37.466351,서울특별시 금천구 가산동 691,금천구,가산동,1154510100,가산동,1154551000
0,126.9110933,37.487519,서울특별시 동작구 신대방동 692-25,동작구,신대방동,1159010900,신대방제1동,1159067000
0,127.026333,37.5114869,서울특별시 강남구 논현동 136-18,강남구,논현동,1168010800,논현1동,1168052100
0,127.0349742,37.5172292,서울특별시 강남구 논현동 71-2,강남구,논현동,1168010800,논현2동,1168053100


In [71]:
lib_df['dong2'].iloc[299] == ' '

True

# 행정동 좌표랑 병합

In [ ]:
addr = pd.read_excel('data/행정_법정동 중심좌표.xlsx')
addr2 = addr[addr['시도'] == '서울특별시']
addr3 = addr2.drop_duplicates(['시군구','읍면동'])

In [75]:
lib_notnull = lib_df[lib_df['dong2'] != ' ']
lib_null = lib_df[lib_df['dong2'] == ' ']

In [82]:
lib_notnull2 = pd.merge(lib_notnull, addr3, left_on = ['city','dong2'], right_on = ['시군구','읍면동'], how = 'left')
lib_null2 = pd.merge(lib_null, addr3, left_on = ['city','dong1'], right_on = ['시군구','읍면동'], how = 'left')

In [91]:
lib_notnull2.index = range(len(lib_notnull2))
lib_notnull3 = lib_notnull2.iloc[lib_notnull2.isnull()[lib_notnull2.isnull()['시도'] == False].index.tolist()]

In [94]:
lib_fin_df = pd.concat([lib_notnull3,lib_null2],axis = 0)

In [98]:
lib_fin_df2 = lib_fin_df[['x_coord','y_coord','dong1','dong2','위도','경도']]

In [109]:
lib_fin_df3 = pd.merge(lib_final,lib_fin_df2,on = ['x_coord','y_coord'], how = 'left')

In [112]:
lib_fin_df4 = lib_fin_df3.drop_duplicates(['location'])

In [162]:
seoul = gpd.read_file('seoul_total_dong_polygon.shp')

In [164]:
lib_fin_df4

,location,x_coord,y_coord,gu,whether,dong1,dong2,위도,경도
0,가락몰도서관,127.112,37.493,송파구,general,가락동,가락1동,37.496537,127.105726
1,가온도서관,127.011,37.5564,중구,general,신당동,다산동,37.554398,127.008155
2,갈현1동문화의집,126.917,37.624,은평구,general,갈현동,갈현제1동,37.602766,126.928839
3,강남구립논현도서관,127.037,37.5173,강남구,general,논현동,논현2동,37.517353,127.037164
4,강남구립대치도서관,127.064,37.4954,강남구,general,대치동,대치2동,37.502252,127.064162
...,...,...,...,...,...,...,...,...,...
299,금천고가,126.8867646,37.466351,금천구,smart,가산동,가산동,37.476681,126.883767
300,신대방역,126.9110933,37.487519,관악구,smart,신대방동,신대방제1동,37.512462,126.939485
301,논현1동,127.026333,37.5114869,강남구,smart,논현동,논현1동,37.511498,127.028552
302,삼성2동,127.0349742,37.5172292,강남구,smart,논현동,논현2동,37.517353,127.037164


# 동별

In [206]:
lib_df_fin2

,x_coord,y_coord,total_addr,city,dong1,dong1_code,dong2,dong2_code,whether
0,127.112,37.493,서울특별시 송파구 가락동 600,송파구,가락동,1171010700,가락1동,1171063100,general
0,127.011,37.5564,서울특별시 중구 신당동 425-11,중구,신당동,1114016200,다산동,1114062500,general
0,126.917,37.624,서울특별시 은평구 갈현동 436-51,은평구,갈현동,1138010400,갈현제1동,1138055100,general
0,127.037,37.5173,서울특별시 강남구 논현동 114-24,강남구,논현동,1168010800,논현2동,1168053100,general
0,127.064,37.4954,서울특별시 강남구 대치동 316,강남구,대치동,1168010600,대치2동,1168061000,general
...,...,...,...,...,...,...,...,...,...
0,126.896,37.5264,서울특별시 영등포구 당산동3가 385-1,영등포구,당산동3가,1156011300,당산제1동,1156055000,middle
0,127.047,37.6686,서울특별시 도봉구 방학동 720,도봉구,방학동,1132010600,방학제1동,1132069000,middle
0,127.093,37.6067,서울특별시 중랑구 신내동 662,중랑구,신내동,1126010600,신내2동,1126069000,middle
0,126.859,37.5234,서울특별시 양천구 신정동 957-9,양천구,신정동,1147010100,신정4동,1147065000,middle


In [213]:
ab = pd.merge(lib_df_fin,lib_df_fin2, on =['x_coord','y_coord','whether'], how = 'left')

In [219]:
ab2 = ab.drop_duplicates(['x_coord','y_coord','whether'])

In [221]:
ab3 = ab2[['location','gu','whether','x_coord','y_coord','dong1','dong2']]

convert와 middle가 같으면 초록 + 빨강 합친 색깔
convert만 초록
middle는 빨강
smart, 일반도서관은 점으로 / convert와 middle는 영역으로

위의 방안들이 안되면 색깔로 middle와 convert만 구분할 수 있도록
따로 구한다음에 이중축으로 하면 될 것 같은데

In [222]:
ab3

,location,gu,whether,x_coord,y_coord,dong1,dong2
0,가락몰도서관,송파구,general,127.112,37.493,가락동,가락1동
1,가온도서관,중구,general,127.011,37.5564,신당동,다산동
2,갈현1동문화의집,은평구,general,126.917,37.624,갈현동,갈현제1동
3,강남구립논현도서관,강남구,general,127.037,37.5173,논현동,논현2동
4,강남구립대치도서관,강남구,general,127.064,37.4954,대치동,대치2동
...,...,...,...,...,...,...,...
349,당산1동,영등포구,middle,126.896,37.5264,당산동3가,당산제1동
350,도봉1동,도봉구,middle,127.047,37.6686,방학동,방학제1동
351,망우3동,중랑구,middle,127.093,37.6067,신내동,신내2동
352,신정4동,양천구,middle,126.859,37.5234,신정동,신정4동


In [225]:
# 겹치는 동들이 많아서 구분하는건 안되겠는데 점으로 해야되겠다
ab3['dong1'].value_counts().values

array([10, 10,  9,  9,  8,  6,  6,  6,  6,  6,  6,  5,  5,  5,  5,  5,  5,
        5,  4,  4,  4,  4,  4,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1])

In [228]:
ab3.groupby(['gu','dong1'])['location'].count().reset_index()

,gu,dong1,location
0,강남구,개포동,3
1,강남구,논현동,4
2,강남구,대치동,5
3,강남구,도곡동,2
4,강남구,삼성동,3
...,...,...,...
159,중랑구,면목동,2
160,중랑구,묵동,1
161,중랑구,상봉동,1
162,중랑구,신내동,2
